# Citi Bike Feature Engineering & Modelling

Creating a model for the probability of each target station for a ride started at "W 21 St & 6 Ave"

## Imports/Loading Data

In [1]:
from __future__ import annotations

# Data manipulation
import numpy as np                                   
import pandas as pd                                  
from pandas.tseries.holiday import USFederalHolidayCalendar  

# Visualization
import matplotlib.pyplot as plt                       
import seaborn as sns                                 

# Machine learning - Preprocessing
from sklearn.compose import ColumnTransformer       
from sklearn.preprocessing import StandardScaler, OneHotEncoder  
from sklearn.pipeline import Pipeline
from sklearn.model_selection import PredefinedSplit

# Machine learning - Models
from sklearn.linear_model import LogisticRegression  

# Machine learning - Metrics
from sklearn.metrics import (
    accuracy_score,                                   
    f1_score,                                        
    classification_report,                           
    log_loss,                                         
)

# Machine learning - Model tuning
from sklearn.model_selection import GridSearchCV

# Utilities
from pathlib import Path                              

In [2]:
pd.set_option("display.max_rows", 20)      
pd.set_option("display.width", 200)
pd.set_option("display.max_colwidth", 80)

In [3]:
SEED = 42

In [4]:
OUT_DIR  = Path(r"..\data\processed")

#cleaned citi bike trip data
df_citibike = pd.read_pickle(OUT_DIR / "citibike_clean.pkl")

In [5]:
#load only relevant trips
df = df_citibike.loc[
    df_citibike["start_station_name"] == "W 21 St & 6 Ave"
].copy()
del df_citibike

In [6]:
#sanity check
print(len(df))
df.head()

5069


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,source_file
223,CCD975D769ADE322,electric_bike,2024-11-11 17:30:25.797,2024-11-11 17:43:27.066,W 21 St & 6 Ave,6140.05,West St & Chambers St,5329.03,40.741741,-73.994156,40.717548,-74.013222,casual,202411-citibike-tripdata_1.csv
560,205492294BE628A1,electric_bike,2024-11-13 08:52:50.918,2024-11-13 08:56:21.928,W 21 St & 6 Ave,6140.05,W 20 St & 10 Ave,6306.01,40.741741,-73.994156,40.745686,-74.005142,member,202411-citibike-tripdata_1.csv
613,9734F97C0880964B,electric_bike,2024-11-12 14:38:48.280,2024-11-12 14:47:07.932,W 21 St & 6 Ave,6140.05,Washington Pl & Greene St,5755.15,40.741741,-73.994156,40.729805,-73.995461,member,202411-citibike-tripdata_1.csv
658,B0556AC58559D864,classic_bike,2024-11-10 17:13:33.843,2024-11-10 17:19:08.497,W 21 St & 6 Ave,6140.05,W 13 St & 7 Ave,6030.04,40.741741,-73.994156,40.737816,-73.999947,member,202411-citibike-tripdata_1.csv
1264,D8C2866285BA2EB1,classic_bike,2024-11-07 19:21:25.445,2024-11-07 19:28:44.325,W 21 St & 6 Ave,6140.05,W 13 St & 5 Ave,5947.04,40.741741,-73.994156,40.735443,-73.994308,member,202411-citibike-tripdata_1.csv


## Feature Engineering

In [7]:
TOP_K = 50  

top_destinations = (
    df["end_station_name"]
    .value_counts()
    .head(TOP_K)
    .index
)

df["y"] = np.where(
    df["end_station_name"].isin(top_destinations),
    df["end_station_name"],
    "OTHER",
)

df["y"].value_counts().head(50)

y
OTHER                      2699
9 Ave & W 22 St             174
W 22 St & 10 Ave            144
W 15 St & 7 Ave              83
W 20 St & 8 Ave              80
                           ... 
E 15 St & 5 Ave              27
E 15 St & 3 Ave              27
W 16 St & The High Line      27
W 30 St & 8 Ave              27
Hudson St & W 13 St          26
Name: count, Length: 50, dtype: int64

for normal stations:
- max percentage 174/5069 = 3,4%
- min percentage 26/5069 = 0,5%

In [8]:
# time based features

ts = df["started_at"]

df["hour"] = ts.dt.hour
df["day_of_week"] = ts.dt.dayofweek  # 0=Mon
df["day_of_year"] = ts.dt.dayofyear
df["month"] = ts.dt.month

# Linear time trend (days since first observation)
df["day_index"] = (
    ts.dt.normalize() -
    ts.dt.normalize().min()
).dt.days

# Weekend flag
df["is_weekend"] = (df["day_of_week"] >= 5).astype(int)

# Rush hour: weekdays 7–10 and 16–19
df["rush_hour"] = (
    (df["day_of_week"] < 5)
    & (
        ((df["hour"] >= 7) & (df["hour"] <= 10))
        | ((df["hour"] >= 16) & (df["hour"] <= 19))
    )
).astype(int)

# Cyclical encodings
df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)

df["doy_sin"] = np.sin(2 * np.pi * df["day_of_year"] / 365)
df["doy_cos"] = np.cos(2 * np.pi * df["day_of_year"] / 365)

In [9]:
# holiday flag

cal = USFederalHolidayCalendar()

holidays = cal.holidays(
    start=df["started_at"].min(),
    end=df["started_at"].max(),
)

df["is_holiday"] = df["started_at"].dt.normalize().isin(holidays).astype(int)
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,day_of_year,month,day_index,is_weekend,rush_hour,hour_sin,hour_cos,doy_sin,doy_cos,is_holiday
223,CCD975D769ADE322,electric_bike,2024-11-11 17:30:25.797,2024-11-11 17:43:27.066,W 21 St & 6 Ave,6140.05,West St & Chambers St,5329.03,40.741741,-73.994156,...,316,11,10,0,1,-0.965926,-0.258819,-0.746972,0.664855,1
560,205492294BE628A1,electric_bike,2024-11-13 08:52:50.918,2024-11-13 08:56:21.928,W 21 St & 6 Ave,6140.05,W 20 St & 10 Ave,6306.01,40.741741,-73.994156,...,318,11,12,0,1,0.866025,-0.500000,-0.723644,0.690173,0
613,9734F97C0880964B,electric_bike,2024-11-12 14:38:48.280,2024-11-12 14:47:07.932,W 21 St & 6 Ave,6140.05,Washington Pl & Greene St,5755.15,40.741741,-73.994156,...,317,11,11,0,0,-0.500000,-0.866025,-0.735417,0.677615,0
658,B0556AC58559D864,classic_bike,2024-11-10 17:13:33.843,2024-11-10 17:19:08.497,W 21 St & 6 Ave,6140.05,W 13 St & 7 Ave,6030.04,40.741741,-73.994156,...,315,11,9,1,0,-0.965926,-0.258819,-0.758306,0.651899,0
1264,D8C2866285BA2EB1,classic_bike,2024-11-07 19:21:25.445,2024-11-07 19:28:44.325,W 21 St & 6 Ave,6140.05,W 13 St & 5 Ave,5947.04,40.741741,-73.994156,...,312,11,6,0,1,-0.965926,0.258819,-0.790946,0.611886,0


In [10]:
#feature matrix -> X,y

feature_cols_num = [
    "hour_sin",
    "hour_cos",
    "doy_sin",
    "doy_cos",
    "day_index",
    "is_weekend",
    "rush_hour",
    "is_holiday"
]

feature_cols_cat = [
    "member_casual",
    "rideable_type"
]

X = df[feature_cols_num + feature_cols_cat]
y = df["y"]

In [11]:
# hold-out last month split + validation month

df["year_month"] = df["started_at"].dt.to_period("M")

last_month = df["year_month"].max()
val_month  = last_month - 1  # previous month

train_idx = df["year_month"] < val_month
val_idx   = df["year_month"] == val_month
test_idx  = df["year_month"] == last_month

X_train, X_val, X_test = X.loc[train_idx], X.loc[val_idx], X.loc[test_idx]
y_train, y_val, y_test = y.loc[train_idx], y.loc[val_idx], y.loc[test_idx]

print("Train size:", X_train.shape[0])
print("Val size:  ", X_val.shape[0],  f"(month={val_month})")
print("Test size: ", X_test.shape[0], f"(month={last_month})")

X_train.head()

Train size: 3966
Val size:   582 (month=2025-09)
Test size:  521 (month=2025-10)


,hour_sin,hour_cos,doy_sin,doy_cos,day_index,is_weekend,rush_hour,is_holiday,member_casual,rideable_type
223,-0.965926,-0.258819,-0.746972,0.664855,10,0,1,1,casual,electric_bike
560,0.866025,-0.500000,-0.723644,0.690173,12,0,1,0,member,electric_bike
613,-0.500000,-0.866025,-0.735417,0.677615,11,0,0,0,member,electric_bike
658,-0.965926,-0.258819,-0.758306,0.651899,9,1,0,0,member,classic_bike
1264,-0.965926,0.258819,-0.790946,0.611886,6,0,1,0,member,classic_bike


## Model

In [12]:
# define pipeline (preprocessing + model)

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), feature_cols_num),
        ("cat", OneHotEncoder(handle_unknown="ignore"), feature_cols_cat),
    ]
)

model = LogisticRegression(
    max_iter=2000,
    random_state = SEED
)

pipe = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("model", model),
    ]
)

We use an unweighted (unbalanced) multiclass logistic regression because this corresponds to maximum-likelihood estimation under the true class distribution. This ensures that the a priori class frequencies are properly incorporated into parameter estimation and that the predicted probabilities are directly calibrated to the real base rates. Balanced weighting would artificially overemphasize rare classes and systematically distort the resulting posterior probabilities (i.e., you would have to recalibrate afterward).

In [13]:
param_grid = {
    "model__C": [0.01, 0.1, 1.0, 10.0, 100.0],
    "model__solver": ["lbfgs"],
}

X_gs = pd.concat([X_train, X_val], axis=0)
y_gs = pd.concat([y_train, y_val], axis=0)

In [14]:
test_fold = np.r_[np.full(len(X_train), -1), np.zeros(len(X_val), dtype=int)]
ps = PredefinedSplit(test_fold)

gs = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="neg_log_loss",   #important
    cv=ps,
    refit=True,
    n_jobs=-1,
    error_score="raise",   
)

gs.fit(X_gs, y_gs)

print("Best params:", gs.best_params_)
print("Val NLL:", -gs.best_score_)

best_pipe = gs.best_estimator_

Best params: {'model__C': 0.01, 'model__solver': 'lbfgs'}
Val NLL: 2.3064425807818503


In [15]:
missing_in_train = set(y_val.unique()) - set(y_train.unique())
print("Classes in val but not in train:", len(missing_in_train))
list(sorted(missing_in_train))[:10]

Classes in val but not in train: 0


[]

***Evaluation focus***

Since the objective is to assess the quality of the predicted class posterior probabilities rather than exact class predictions, the evaluation focuses on probability-based metrics and calibration instead of precision, recall, or F1 score.


In [16]:
probs_test = best_pipe.predict_proba(X_test)
classes = best_pipe.classes_
n_test = len(y_test)

print("Test samples:", n_test)
print("Num classes:", len(classes))

Test samples: 521
Num classes: 51


In [17]:
# Baseline probabilities = empirical class distribution in training set
model_nll = log_loss(y_test, probs_test, labels=classes)
train_freq = y_train.value_counts(normalize=True)

baseline_probs_row = np.array([train_freq.get(c, 0.0) for c in classes])
baseline_probs = np.tile(baseline_probs_row, (n_test, 1))

baseline_nll = log_loss(y_test, baseline_probs, labels=classes)

print("Baseline log loss (NLL):", baseline_nll)
print("Model log loss (NLL):", model_nll)
print("ΔNLL (baseline - model):", baseline_nll - model_nll)

Baseline log loss (NLL): 2.4286250087795027
Model log loss (NLL): 2.397483370929303
ΔNLL (baseline - model): 0.031141637850199633


**Analysis**

The model performs almost the same as the baseline which consists of the frequencies of each class in the training data. This could be for several reasons:
- limited trainingdata (retry with more years and full data for each year, not 3%)
- maybe destination choice is close to independent from temporal/bybcicle type/membership (our features)
- maybe relationship between features and destination choice is non linear (logstic regression can only modell linear decision boundaries)

**Outlook**

- retry with more years and full data for each year, not 3% of the data of one year
- add new features like spatial information (distance, bearing, or destination clusters), weather conditions, or historical destination popularity conditional on time (how popular was destination xy one hour/one day ago
- try more complex models
- group stations into spatial zones and predict these zones (this would be ok for our final pricing model)

